# LOCA2 Inventory Checks

## Libraries

In [1]:
######################################################
#
# Libraries
#


import os    as os

import numpy as np

import xarray as xr

import pandas as pd

import matplotlib.pyplot as plt

#
######################################################

## File Control

In [2]:
######################################################
#
# File Control
#

working_directory  = "./"

thredds_root_dir   = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/"


loca2_mod_member_file = "LOCA2_Model_Member_Complete_List.csv"




Scenarios = ["historical",
                 "ssp245",
                 "ssp370",
                 "ssp585"]

Vars = ["tasmax",
        "tasmin",
            "pr"]

#
######################################################

## Pull All Possible Models and Members

In [3]:
######################################################
#
# Pull Full Inventory List
#

full_models_and_members_list = pd.read_csv(filepath_or_buffer = loca2_mod_member_file)

full_models_and_members_list

n_records = len(full_models_and_members_list)

display(full_models_and_members_list)

display(n_records)

#
######################################################

,Model,Member,Var,Season,Stat,NetError,Rank,historical,ssp245,ssp370,ssp585
0,GFDL-ESM4,r1i1p1f1,All,Annual,Both,6.152235,1,XNP,XNP,XNP,XNP
1,BCC-CSM2-MR,r1i1p1f1,All,Annual,Both,6.357673,1,XNP,XNP,XNP,XNP
2,GFDL-CM4,r1i1p1f1,All,Annual,Both,6.498461,1,XNP,XNP,---,XNP
3,CNRM-CM6-1,r1i1p1f2,All,Annual,Both,6.513064,1,XNP,XNP,XNP,XNP
4,TaiESM1,r1i1p1f1,All,Annual,Both,6.658829,1,XNP,XNP,XNP,---
...,...,...,...,...,...,...,...,...,...,...,...
103,MPI-ESM1-2-HR,r8i1p1f1,All,Annual,Both,13.374229,9,NaN,---,NaN,---
104,MPI-ESM1-2-LR,r4i1p1f1,All,Annual,Both,14.062717,9,NaN,NaN,NaN,NaN
105,CESM2-LENS,r8i1p1f1,All,Annual,Both,12.450703,10,NaN,---,NaN,---
106,IPSL-CM6A-LR,r7i1p1f1,All,Annual,Both,13.239335,10,NaN,---,NaN,---


108

In [4]:
loca2_mod_member_file

'LOCA2_Model_Member_Complete_List.csv'

## Scan Disk for Available Files

In [5]:
######################################################
#
# Scan Disk for Available Files
#

available_rows =  [False for i in range(len(full_models_and_members_list))]


for i in range(len(full_models_and_members_list)):
    
    for k in range(len(Scenarios)):
    
        file_name  = "LOCA2-CONUS___" + full_models_and_members_list.at[i,"Model"]+"."+ full_models_and_members_list.at[i,"Member"]+"___"+Scenarios[k]+".nc"

        #print(file_name)

        file_name  = thredds_root_dir + Scenarios[k] + "/" + file_name

        if os.path.isfile(file_name):

            my_string_all = "---"
            my_string_max = "-"
            my_string_min = "-"
            my_string_pcp = "-"
            xf         = xr.open_dataset(filename_or_obj = file_name)

            if "tasmax" in list(xf.keys()):
                my_string_max = "X"
                available_rows[i] = True
            if "tasmin" in list(xf.keys()):
                my_string_min = "N"        
                available_rows[i] = True
            if "pr"     in list(xf.keys()):
                my_string_pcp = "P"    
                available_rows[i] = True
            full_models_and_members_list.at[i,  Scenarios[k]] = my_string_max+my_string_min+my_string_pcp

            


full_models_and_members_list.sort_values(by           = ["Rank","NetError"],
                                         inplace      =                True,
                                         ignore_index =                True)
        
#
######################################################

## Save Data 

In [6]:
######################################################
#
# Save Full Inventory
#

full_models_and_members_list.to_csv(path_or_buf = working_directory + loca2_mod_member_file,
                                   index        =  False,
                                   na_rep       =     '')

#
######################################################

# Cut only available files

In [7]:
######################################################
#
# Trim Only Available Ones
#

avail_models_and_members_list = full_models_and_members_list.copy()

avail_models_and_members_list = avail_models_and_members_list[available_rows]

avail_models_and_members_list.sort_values(by           = ["Rank","NetError"],
                                          inplace      =                True,
                                          ignore_index =                True)
                                                              
                                                              
display(avail_models_and_members_list)

avail_models_and_members_list.to_csv(path_or_buf = working_directory + "LOCA2_Model_Member_Available_List.csv",
                                    index        =  False,
                                    na_rep       =     '')

os.system("scp ./LOCA2_Model_Member_Available_List.csv wjc@kyrill:"+thredds_root_dir)
os.system("scp ./LOCA2_Model_Member_Complete_List.csv wjc@kyrill:"+thredds_root_dir)

#
######################################################

,Model,Member,Var,Season,Stat,NetError,Rank,historical,ssp245,ssp370,ssp585
0,GFDL-ESM4,r1i1p1f1,All,Annual,Both,6.152235,1,XNP,XNP,XNP,XNP
1,BCC-CSM2-MR,r1i1p1f1,All,Annual,Both,6.357673,1,XNP,XNP,XNP,XNP
2,GFDL-CM4,r1i1p1f1,All,Annual,Both,6.498461,1,XNP,XNP,---,XNP
3,CNRM-CM6-1,r1i1p1f2,All,Annual,Both,6.513064,1,XNP,XNP,XNP,XNP
4,TaiESM1,r1i1p1f1,All,Annual,Both,6.658829,1,XNP,XNP,XNP,---
...,...,...,...,...,...,...,...,...,...,...,...
60,EC-Earth3-Veg,r1i1p1f1,All,Annual,Both,12.301219,4,XNP,XNP,XNP,XNP
61,MPI-ESM1-2-HR,r1i1p1f1,All,Annual,Both,12.387494,4,XNP,XNP,XNP,XNP
62,CESM2-LENS,r1i1p1f1,All,Annual,Both,11.249889,6,XNP,---,XNP,---
63,IPSL-CM6A-LR,r1i1p1f1,All,Annual,Both,11.812705,6,XNP,XNP,XNP,XNP


0